In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [3]:
def longest_path(T, G, v1, v2, justH1=False):
    if v1 not in G.graph or v2 not in G.graph:
        return None
    temp_longest = {v1 : G.graph[v1].weight}
    
    print G.graph[v1].weight
    
    if v1 == v2:
        return temp_longest[v1]
    
    for v in T[T.index(v1)+1:]:
        if (not justH1) or v[0] == 0:
            print v
            preds = [p for p in G.graph[v].predecessors if p in temp_longest]
            print preds
            if preds:
                best_pred = max(preds, key=lambda p: temp_longest[p] + G.graph[p].weight)
                print best_pred
                temp_longest[v] = temp_longest[best_pred] + G.graph[best_pred].weight
            if v == v2:
                if v2 in temp_longest:
                    return temp_longest[v2]
                return None

In [4]:
# m_from, m_to - skąd zabieramy, gdzie wsadzamy
# op - zabierana operacja, para (nr_maszyny, nr_operacji)
# pos - pozycja do wsadzenia op na maszynę m_to

# tutaj numery maszyn idą od 0
def lower_bound(G, m_from, op, m_to, pos):
    def alpha(k,i):
        assert i >= 0
        if k != m_from or solution[k].index(op) > i:
            return G.solution[k][i]
        return G.solution[k][i+1]
    
    # jeśli longest_path nie istnieje, to powinien zwrócić None
    # te drogi poniżej są zawarte w H1 (pierwszej składowej G), więc
    # można by je liczyć trochę szybciej (tylko w H1)
    
    def R(k,opr):
        return G.longest_path((0,) + alpha(k,0), (0,) + opr, justH1=True)
    
    def Q(k,opr):
        return G.longest_path((0,) + opr, (0,) + alpha(k, len(G.solution[k])-1), justH1=True)

    def LB(l):
        op_old_weight = G.graph[(0,) + op].weight
        op_new_weight = G.problem[op[0]-1][op[1]-1][m_to]
        
        R1 = lambda: R(l, alpha(m_to, pos-1))
        R2 = lambda: R(l, op) - op_old_weight
        Q1 = lambda: Q(l, alpha(m_to, pos))
        Q2 = lambda: Q(l, op) - op_old_weight
        
        # 0 w maxach po to, żeby mieć jakąś liczbę, gdy drogi nie istnieją
        if pos == 0:
            return max(R2(), 0) + max(Q1(), Q2(), 0) + op_old_weight
        if pos == len(G.solution[k]):
            return max(R1(), R2(), 0) + max(Q2(), 0) + op_old_weight
        
        maxR = max(R1(), R2(), 0)
        maxQ = max(Q1(), Q2(), 0)
        
        return maxR + maxQ + op_old_weight # a może tu ma być op_new_weight?
    
    return max([LB(l) for l in xrange(len(solution))])

# JUŻ NIEAKTUALNE:
# co jeśli m_from == m_to (czy w ogóle może tak być?) - alpha(m_to, pos) może nie istnieć

In [5]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors
G.topological_sort()

(1, 5, 5)
[(0, 3, 3), (0, 5, 6)]
[(1, 3, 3), (1, 5, 6)]
[(0, 3, 3), (0, 5, 6)]
(1, 10, 4)
[(0, 6, 8), (0, 10, 5)]
[(1, 6, 8), (1, 10, 5)]
[(0, 6, 8), (0, 10, 5)]
(1, 5, 7)
[(0, 7, 8), (0, 5, 8)]
[(1, 7, 8), (1, 5, 8)]
[(0, 7, 8), (0, 5, 8)]
(1, 10, 6)
[(0, 5, 9), (0, 10, 7)]
[(1, 5, 9), (1, 10, 7)]
[(0, 5, 9), (0, 10, 7)]
(1, 2, 1)
[(0, 7, 2), (0, 2, 2)]
[(1, 7, 2), (1, 2, 2)]
[(0, 7, 2), (0, 2, 2)]
(1, 5, 1)
[(0, 8, 1), (0, 5, 2)]
[(1, 8, 1), (1, 5, 2)]
[(0, 8, 1), (0, 5, 2)]
(1, 7, 10)
[(0, 9, 9)]
[(1, 9, 9)]
[(0, 9, 9)]
(1, 10, 8)
[(0, 4, 8), (0, 10, 9)]
[(1, 4, 8), (1, 10, 9)]
[(0, 4, 8), (0, 10, 9)]
(1, 2, 3)
[(0, 5, 6), (0, 2, 4)]
[(1, 5, 6), (1, 2, 4)]
[(0, 5, 6), (0, 2, 4)]
(1, 4, 10)
[]
[]
[]
(1, 5, 3)
[(0, 3, 1), (0, 5, 4)]
[(1, 3, 1), (1, 5, 4)]
[(0, 3, 1), (0, 5, 4)]
(1, 7, 8)
[(0, 3, 5), (0, 7, 9)]
[(1, 3, 5), (1, 7, 9)]
[(0, 3, 5), (0, 7, 9)]
(1, 10, 10)
[]
[]
[]
(1, 2, 5)
[(0, 10, 8), (0, 2, 6)]
[(1, 10, 8), (1, 2, 6)]
[(0, 10, 8), (0, 2, 6)]
(1, 4, 8)
[(0, 8, 10), (0, 4

In [6]:
G.graph

{(11, 5, 9): <graph_representation.Node at 0x7f94acfc0d90>,
 (0, 5, 5): <graph_representation.Node at 0x7f94b4115310>,
 (1, 2, 8): <graph_representation.Node at 0x7f94b40cb310>,
 (10, 3, 3): <graph_representation.Node at 0x7f94b4064610>,
 (0, 10, 4): <graph_representation.Node at 0x7f94b4115410>,
 (1, 4, 4): <graph_representation.Node at 0x7f94b40c4f50>,
 (4, 10, 5): <graph_representation.Node at 0x7f94b407e0d0>,
 (10, 2, 8): <graph_representation.Node at 0x7f94b4064790>,
 (8, 9, 1): <graph_representation.Node at 0x7f94b4037d50>,
 (4, 7, 6): <graph_representation.Node at 0x7f94b40f36d0>,
 (7, 5, 10): <graph_representation.Node at 0x7f94b40abbd0>,
 (9, 8, 3): <graph_representation.Node at 0x7f94b4058710>,
 (10, 1, 6): <graph_representation.Node at 0x7f94b4064550>,
 (2, 8, 2): <graph_representation.Node at 0x7f94b40d1910>,
 (7, 10, 8): <graph_representation.Node at 0x7f94b40ab110>,
 (5, 2, 1): <graph_representation.Node at 0x7f94b407ed50>,
 (0, 2, 1): <graph_representation.Node at 0x7f94

In [8]:
G.graph[(11,9,9)].successors

[(11, 10, 9), (11, 9, 10)]

In [9]:
len(G.topological_order)

1200

In [59]:
longest_path(G.topological_order, G, (0,9,1), (0,2,1))

76
(1, 7, 6)
[]
(1, 10, 1)
[]
(1, 1, 5)
[]
(1, 3, 3)
[]
(1, 4, 8)
[]
(1, 9, 8)
[]
(1, 6, 9)
[]
(1, 3, 4)
[]
(1, 2, 3)
[]
(1, 8, 3)
[]
(1, 5, 8)
[]
(1, 10, 8)
[]
(1, 7, 7)
[]
(1, 1, 2)
[]
(1, 9, 9)
[]
(1, 1, 10)
[]
(0, 6, 1)
[]
(1, 6, 8)
[]
(1, 3, 8)
[]
(1, 8, 10)
[]
(1, 5, 1)
[]
(1, 8, 9)
[]
(1, 3, 5)
[]
(1, 3, 10)
[]
(1, 2, 2)
[]
(1, 8, 2)
[]
(1, 5, 9)
[]
(1, 10, 7)
[]
(1, 4, 2)
[]
(1, 2, 10)
[]
(1, 7, 8)
[]
(1, 1, 3)
[]
(1, 4, 10)
[]
(1, 8, 8)
[]
(1, 9, 6)
[]
(1, 6, 7)
[]
(1, 10, 9)
[]
(1, 8, 5)
[]
(1, 5, 6)
[]
(1, 3, 6)
[]
(0, 4, 1)
[]
(1, 2, 1)
[]
(1, 10, 6)
[]
(1, 7, 1)
[]
(1, 4, 5)
[]
(1, 2, 9)
[]
(1, 7, 9)
[]
(1, 9, 7)
[]
(1, 1, 8)
[]
(1, 8, 4)
[]
(1, 5, 7)
[]
(1, 3, 7)
[]
(1, 10, 5)
[]
(1, 7, 2)
[]
(1, 8, 1)
[]
(1, 2, 8)
[]
(2, 4, 1)
[]
(2, 9, 1)
[]
(2, 6, 1)
[]
(2, 2, 1)
[]
(2, 4, 2)
[]
(2, 6, 2)
[]
(2, 5, 1)
[]
(2, 4, 3)
[]
(2, 7, 1)
[]
(2, 6, 3)
[]
(2, 8, 1)
[]
(2, 4, 4)
[]
(2, 10, 1)
[]
(2, 7, 2)
[]
(2, 6, 4)
[]
(2, 2, 2)
[]
(2, 8, 2)
[]
(2, 9, 2)
[]
(2, 10, 2)
[]
(2, 7, 3)

In [35]:
G.problem[8][1]

{2: 69}